<a href="https://colab.research.google.com/github/aurioldegbelo/sis2025/blob/main/2025_SIS_Demo_2_DataFrameQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Topics: Data Modelling and Search Models
* Question anwering over pandas data frames

# Part 1: Data Preparation

In [ ]:
import requests
import pandas as pd
import json

# Get table data
def get_table_data(url):

    response = requests.get(url)

    # store all tables in the tables list
    tables = []
    geometry_data = response.json()
    # loop through the dataset and convert tabular data to pandas dataframes
    for doc in geometry_data:
        table = pd.DataFrame(doc["data"], columns=doc["header"])
        tables.append(table)

    return tables, geometry_data

tables, geometry_data = get_table_data("https://raw.githubusercontent.com/aurioldegbelo/sis2025/refs/heads/main/vector_data/data_table.json")
print(len(tables))
print(type(tables[0]))
tables[0]

2
<class 'pandas.core.frame.DataFrame'>


,id,type,name,population,area,state_capital,geometry
0,s01,State,Nordrhein-Westfalen,18139116,34110.26,Düsseldorf,nrw_geom
1,s02,State,Hessen,6391360,21114.94,Wiesbaden,hessen_geom
2,s03,State,Rheinland-Pfalz,4159150,19854.21,Mainz,rlp_geom
3,s04,State,Niedersachsen,8140242,47709.82,Hannover,niedersachsen_geom


In [ ]:
tables[1]

,id,type,touches,url
0,nrw_geom,Geometry,"[hessen_geom, rlp_geom, niedersachsen_geom]",https://github.com/aurioldegbelo/sis2025/blob/...
1,hessen_geom,Geometry,"[nrw_geom, rlp_geom, niedersachsen_geom]",https://github.com/aurioldegbelo/sis2025/blob/...
2,rlp_geom,Geometry,"[nrw_geom, hessen_geom]",https://github.com/aurioldegbelo/sis2025/blob/...
3,niedersachsen_geom,Geometry,"[hessen_geom, nrw_geom]",https://github.com/aurioldegbelo/sis2025/blob/...


# Part 2: QA using [Langchain](https://www.langchain.com/)


In [ ]:
!pip install --quiet langchain langchain_openai langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [ ]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


One data frame

In [ ]:
df = tables[0]

# https://python.langchain.com/docs/integrations/tools/pandas/
# an agent is a system that uses an LLM to decide the control flow of an application, see https://langchain-ai.github.io/langgraph/concepts/high_level/
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    df,
    verbose=True,
    allow_dangerous_code=True
)

In [ ]:
question1 = "How many states in the database?"
question2 = "How many geometries in the the database?"
question3 = "What is the population of Hessen?"
question4 = "What is the area of Hessen?"
question5 = "What is the capital of Hessen?"
question6 = "What is the geometry of Hessen?"
question7 = "What are the geometries of Hessen and Niedersachsen?"
question8 = "What is the url of the geometry of Hessen?"

agent.invoke(question8)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to get the row corresponding to Hessen and then extract the geometry url.
Action: python_repl_ast
Action Input: df[df['name'] == 'Hessen']['geometry'].values[0]hessen_geomI now know the final answer
Final Answer: hessen_geom

> Finished chain.


{'input': 'What is the url of the geometry of Hessen?',
 'output': 'hessen_geom'}

Several dataframes

In [ ]:
tables

[    id   type                 name  population      area state_capital  \
 0  s01  State  Nordrhein-Westfalen    18139116  34110.26    Düsseldorf   
 1  s02  State               Hessen     6391360  21114.94     Wiesbaden   
 2  s03  State      Rheinland-Pfalz     4159150  19854.21         Mainz   
 3  s04  State        Niedersachsen     8140242  47709.82      Hannover   
 
              geometry  
 0            nrw_geom  
 1         hessen_geom  
 2            rlp_geom  
 3  niedersachsen_geom  ,
                    id      type                                      touches  \
 0            nrw_geom  Geometry  [hessen_geom, rlp_geom, niedersachsen_geom]   
 1         hessen_geom  Geometry     [nrw_geom, rlp_geom, niedersachsen_geom]   
 2            rlp_geom  Geometry                      [nrw_geom, hessen_geom]   
 3  niedersachsen_geom  Geometry                      [hessen_geom, nrw_geom]   
 
                                                  url  
 0  https://github.com/aurioldegbe

In [ ]:
all_df = tables

agent2 = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4o-mini"), # gpt-4o-mini	gpt-3.5-turbo
    all_df,
    verbose=True,
    allow_dangerous_code=True
)

In [ ]:
question1 = "How many states in the database?"
question2 = "How many geometries in the the database?"
question3 = "What is the population of Hessen?"
question4 = "What is the area of Hessen?"
question5 = "What is the capital of Hessen?"
question6 = "What is the geometry of Hessen?"
question7 = "What are the geometries of Hessen and Niedersachsen?"
question8 = "What is the url of the geometry of Hessen?"

question9 = "Which state(s) touch Hessen?"
question10 = "What is the ulr of nrw_geom?"


agent2.invoke(question3)



> Entering new AgentExecutor chain...
Thought: To find the population of Hessen, I need to look for the row in `df1` where the name is "Hessen" and then extract the population value from that row. 

Action: I will filter `df1` for the row where the name is "Hessen" and retrieve the population value. 
Action Input: `df1[df1['name'] == 'Hessen']['population'].values[0]`I will filter `df1` for the row where the name is "Hessen" and retrieve the population value. is not a valid tool, try one of [python_repl_ast].Thought: To find the population of Hessen, I need to look for the row in `df1` where the name is "Hessen" and then extract the population value from that row. 

Action: I will filter `df1` for the row where the name is "Hessen" and retrieve the population value. 
Action Input: `df1[df1['name'] == 'Hessen']['population'].values[0]`I will filter `df1` for the row where the name is "Hessen" and retrieve the population value. is not a valid tool, try one of [python_repl_ast].To find 

{'input': 'What is the population of Hessen?',
 'output': 'Agent stopped due to iteration limit or time limit.'}